In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

In [2]:
import copy
import json
import os

import numpy as np

from pycocotools.coco import COCO
from sam3.train.eval.demo_eval import DemoEvaluator

/data/home/shoubhikdn/miniconda/envs/sam3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enabled the use of perflib.


In [3]:
# Update to the directory where the GT annotation and PRED files exist
GT_DIR = "/fsx-onevision/shoubhikdn/release/roboflow/gt-annotations/"  # PUT YOUR PATH HERE
PRED_DIR = "/fsx-onevision/shoubhikdn/release/sam3_predictions/arxiv/" # PUT YOUR PATH HERE

In [4]:
# Relative file names for GT and prediction files for 7 SA-Co/Gold subsets
saco_gold_gt_and_pred_files = {
    # MetaCLIP Captioner
    "metaclip": {
        "gt_fname": [
            "gold_metaclip_merged_a_release_test.json",
            "gold_metaclip_merged_b_release_test.json",
            "gold_metaclip_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_metaclip_captioner.json",
    },
    # SA-1B captioner
    "sa1b": {
        "gt_fname": [
            "gold_sa1b_merged_a_release_test.json",
            "gold_sa1b_merged_b_release_test.json",
            "gold_sa1b_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_sa1b_captioner.json",
    },
    # Crowded
    "crowded": {
        "gt_fname": [
            "gold_crowded_merged_a_release_test.json",
            "gold_crowded_merged_b_release_test.json",
            "gold_crowded_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_crowded.json",
    },
    # FG Food
    "fg_food": {
        "gt_fname": [
            "gold_fg_food_merged_a_release_test.json",
            "gold_fg_food_merged_b_release_test.json",
            "gold_fg_food_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_fg_food.json",
    },
    # FG Sports
    "fg_sports_equipment": {
        "gt_fname": [
            "gold_fg_sports_equipment_merged_a_release_test.json",
            "gold_fg_sports_equipment_merged_b_release_test.json",
            "gold_fg_sports_equipment_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_fg_sports_equipment.json",
    },
    # Attributes
    "attributes": {
        "gt_fname": [
            "gold_attributes_merged_a_release_test.json",
            "gold_attributes_merged_b_release_test.json",
            "gold_attributes_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_attr.json",
    },
    # Wiki common
    "wiki_common": {
        "gt_fname": [
            "gold_wiki_common_merged_a_release_test.json",
            "gold_wiki_common_merged_b_release_test.json",
            "gold_wiki_common_merged_c_release_test.json",
        ],
        "pred_fname": "coco_predictions_gold_wiki_common.json",
    },
}

In [5]:
class PostProcessorMerged:
    def __init__(self, predictions_path, img_ids):
        with open(predictions_path, "r") as f:
            self.data_anns = json.load(f)
        self.img_ids = img_ids

    def process_results(self):
        d = {
            "scores": np.array([]),
            "labels": np.array([], dtype=int),
            "masks_rle": [],
        }
        imgToAnns = {img_id: copy.deepcopy(d) for img_id in self.img_ids}
        for ann in self.data_anns:
            if ann["image_id"] not in self.img_ids:
                continue
            imgToAnns[ann["image_id"]]["scores"] = np.append(
                imgToAnns[ann["image_id"]]["scores"], ann["score"]
            )
            imgToAnns[ann["image_id"]]["labels"] = np.append(
                imgToAnns[ann["image_id"]]["labels"], ann["category_id"]
            )
            imgToAnns[ann["image_id"]]["masks_rle"].append(ann["segmentation"])

        return imgToAnns

<b>Run offline evaluation for all 7 SA-Co/Gold subsets</b>

In [6]:
results = ""

for subset_name, values in saco_gold_gt_and_pred_files.items():
    print("Processing subset: ", subset_name)
    gt_fnames = values["gt_fname"]
    pred_fname = values["pred_fname"]
    gt_fname_full_paths = [
        os.path.join(GT_DIR, gt_fname) for gt_fname in gt_fnames
    ]
    pred_fname_full_path = os.path.join(PRED_DIR, pred_fname)
    coco = COCO(gt_fname_full_paths[0])

    img_ids = list(
        sorted(
            [
                img["id"]
                for img in coco.dataset["images"]
                if img["is_instance_exhaustive"]
            ]
        )
    )

    evaluator = DemoEvaluator(
        coco_gt=gt_fname_full_paths,
        iou_types=["segm"],
        threshold=0.5,
        dump_dir=None,
        postprocessor=PostProcessorMerged(
            predictions_path=pred_fname_full_path, img_ids=img_ids
        ),
        average_by_rarity=False,
        gather_pred_via_filesys=False,
        exhaustive_only=True,
    )
    evaluator.update()
    summary = evaluator.compute_synced()

    cgf1 = str(round(summary["coco_eval_masks_oracle_CGF1"] * 100, 2))
    cgf1m = str(round(summary["coco_eval_masks_oracle_CGF1_micro"] * 100, 2))
    il_mcc = str(round(summary["coco_eval_masks_oracle_IL_MCC"], 2))
    pmf1 = str(round(summary["coco_eval_masks_oracle_Macro_F1"] * 100, 2))
    pmf1m = str(round(summary["coco_eval_masks_oracle_positive_micro_F1"] * 100, 2))
    demof1 = str(round(summary["coco_eval_masks_oracle_F1"] * 100, 2))
    final_str = f"{cgf1},{cgf1m},{il_mcc},{pmf1},{pmf1m},{demof1}"
    results += subset_name + ": " + final_str + "\n"

Processing subset:  metaclip
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
Demo metric, IoU type=segm
 Average CGF1       @[ IoU=0.50:0.95] = 0.612
 Average precision  @[ IoU=0.50:0.95] = 0.598
 Average recall     @[ IoU=0.50:0.95] = 0.543
 Average F1         @[ IoU=0.50:0.95] = 0.569
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.754
 Average IL_precision                = 0.913
 Average IL_recall                   = 0.771
 Average IL_F1                       = 0.836
 Average IL_FPR                      = 0.014
 Average IL_MCC                      = 0.812
 Average IL_perfect_pos @[ IoU=0.50:0.95] = 0.418
 Average IL_perfect_neg @[ IoU=0.50:0.95] = 0.986
 Average CGF1       @[ IoU=0.50     ] = 0.708
 Ave

In [7]:
print("Subset name, CG_F1, CG_F1_m, IL_MCC, pmF1, pmF1_m, demoF1")
print(results)

Subset name, CG_F1, CG_F1_m, IL_MCC, pmF1, pmF1_m, demoF1
metaclip: 61.22,47.82,0.81,75.43,58.93,56.92
sa1b: 64.36,53.9,0.86,74.45,62.35,61.62
crowded: 61.44,60.9,0.9,68.3,67.7,66.01
fg_food: 64.9,54.03,0.8,81.36,67.74,65.49
fg_sports_equipment: 74.93,66.97,0.9,83.34,74.49,72.1
attributes: 66.46,55.28,0.77,86.71,72.13,65.67
wiki_common: 56.57,43.22,0.7,80.44,61.46,58.55

